# 작업형2
- 빅데이터 분석기사 3회 실기
- 제공된 데이터
    - train.csv
    - test.csv
- 예측할 컬럼 : TravelInsurance

### 시험에서 제공된 코드
~~~
import pandas as pd
a = pd.read_csv("train.csv")
b = pd.read_csv('test.csv')

pd.DataFrame(변수).to_csv('000000000.csv')
~~~


### 보험가입 확률을 묻는 문제
~~~
# csv 형태
# index,y_pred
# 0,0.152467
# 1,0.961511
# 2,0.151313
# 3,0.236632
...
# 494,0.121372
# 495,0.965155
# 496,0.986405

~~~


## 데이터 불러오기

In [ ]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/test.csv")

## EDA

In [ ]:
# 크기 확인
print(train.shape)
print(test.shape)

(1490, 10)
(497, 9)


In [ ]:
test.head()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,920,32,Government Sector,Yes,650000,5,0,No,No
1,777,27,Government Sector,Yes,500000,5,0,No,No
2,696,32,Government Sector,Yes,350000,2,0,No,No
3,907,26,Private Sector/Self Employed,Yes,1400000,6,0,No,Yes
4,675,27,Private Sector/Self Employed,No,350000,3,0,No,No


In [ ]:
# 샘플 확인
train.sample()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
721,1591,28,Government Sector,Yes,300000,3,0,No,No,0


In [ ]:
# 타입 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           497 non-null    int64 
 1

In [ ]:
# 결측치 확인 train
train.isnull().sum()

,0
Unnamed: 0,0
Age,0
Employment Type,0
GraduateOrNot,0
AnnualIncome,0
FamilyMembers,0
ChronicDiseases,0
FrequentFlyer,0
EverTravelledAbroad,0
TravelInsurance,0


In [ ]:
# 결측치 확인 test
test.isnull().sum()

,0
Unnamed: 0,0
Age,0
Employment Type,0
GraduateOrNot,0
AnnualIncome,0
FamilyMembers,0
ChronicDiseases,0
FrequentFlyer,0
EverTravelledAbroad,0


In [ ]:
# 기초 통계 획인
train.describe()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
count,1490.000000,1490.000000,1.490000e+03,1490.000000,1490.000000,1490.000000
mean,992.277181,29.600000,9.310738e+05,4.755705,0.280537,0.352349
std,566.637006,2.887829,3.763057e+05,1.603613,0.449412,0.477862
min,0.000000,25.000000,3.000000e+05,2.000000,0.000000,0.000000
25%,502.250000,28.000000,6.000000e+05,4.000000,0.000000,0.000000
50%,994.500000,29.000000,9.000000e+05,5.000000,0.000000,0.000000
75%,1479.750000,32.000000,1.250000e+06,6.000000,1.000000,1.000000
max,1983.000000,35.000000,1.800000e+06,9.000000,1.000000,1.000000


In [ ]:
# 기초 통계 획인 object
train.describe(include='object')

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [ ]:
# 타겟(레이블)
train['TravelInsurance'].value_counts()

,count
TravelInsurance,
0,965
1,525


## 데이터 전처리 및 피처엔지니어링

In [ ]:
# 수치형 데이터와 범주형 데이터 분리
#n은 수치형 데이터 c는 범주형 데이터
n_train = train.select_dtypes(exclude = 'object').copy()
c_train = train.select_dtypes(include = 'object').copy()
n_test = test.select_dtypes(exclude = 'object').copy()
c_test = test.select_dtypes(include = 'object').copy()


#수치형 변수는 스케일링을 하는것이고
#범주형 변수는 인코딩 하는 것임!!!



In [ ]:
# 수치형 변수 스케일링
#RobustScaler 사용한 이유는 이상치에 대해 정규화를 더 잘시키지 않을까 해서 함.
#그리고 수치형 차이가 너무 큰 경우 모델에 지나치게 큰영향을 줘서, FamilyMebers와 annualincome이 차이도 너무크기 때문에
#cols에 TravelInsurance는 타겟값이기 떄문에 수치형 이라도 넣으면 안됨!
#소득값이 더 큰 영향을 끼칠 수 있음 때문에 변수의 값 범위를 비슷하게 만들기 위해 진행
#타겟값은 스케일링 하지 않음
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols = ['Age','AnnualIncome', 'FamilyMembers', 'ChronicDiseases']

display(n_train.head())
n_train[cols] = scaler.fit_transform(n_train[cols])
n_test[cols] = scaler.transform(n_test[cols])
n_train.head()


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
0,888,28,1250000,6,1,0
1,1308,31,1250000,7,1,0
2,151,29,1200000,7,0,1
3,396,33,650000,6,1,1
4,135,28,800000,6,0,1


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
0,888,-0.25,0.538462,0.5,1.0,0
1,1308,0.50,0.538462,1.0,1.0,0
2,151,0.00,0.461538,1.0,0.0,1
3,396,1.00,-0.384615,0.5,1.0,1
4,135,-0.25,-0.153846,0.5,0.0,1


In [ ]:
# 범주형 변수 인코딩
# 원핫 인코딩 진행
#0과 1로 만들어주려면 기존의 true/false값을 0과 1로 만들어 주는건데, 이게 안될경우 astype(int)를 사용해서 정수형 숫자로 만들어주기
display(c_train.head())
c_train = pd.get_dummies(c_train).astype(int)
c_test = pd.get_dummies(c_test).astype(int)
c_train.head()



,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,False,True,False,True,True,False,True,False
1,False,True,False,True,True,False,True,False
2,False,True,False,True,True,False,True,False
3,True,False,False,True,True,False,True,False
4,False,True,False,True,True,False,False,True


,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,0,1,0,1,1,0,1,0
1,0,1,0,1,1,0,1,0
2,0,1,0,1,1,0,1,0
3,1,0,0,1,1,0,1,0
4,0,1,0,1,1,0,0,1


In [ ]:
# 분리한 데이터 다시 합침
#스케일링과 원핫 인코딩한 수치형 데이터와 범주형 데이터를 다시 합치기 concat사용해서
train = pd.concat([n_train, c_train], axis=1)
test = pd.concat([n_test,c_test], axis=1)
print(train.shape, test.shape)
train.head()

(1490, 14) (497, 13)


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,888,-0.25,0.538462,0.5,1.0,0,0,1,0,1,1,0,1,0
1,1308,0.50,0.538462,1.0,1.0,0,0,1,0,1,1,0,1,0
2,151,0.00,0.461538,1.0,0.0,1,0,1,0,1,1,0,1,0
3,396,1.00,-0.384615,0.5,1.0,1,1,0,0,1,1,0,1,0
4,135,-0.25,-0.153846,0.5,0.0,1,0,1,0,1,1,0,0,1


## 검증데이터 분리

In [ ]:
# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train.drop('TravelInsurance', axis=1),
                                            train['TravelInsurance'],
                                            test_size=0.1,
                                            random_state=1204)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((1341, 13), (149, 13), (1341,), (149,))

## 모델 학습 및 예측

In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=400, max_depth=9, random_state=1204)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)[:,1]


In [ ]:
# 평가
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred)

0.8277551020408163

## 예측

In [ ]:
# test 예측
pred = rf.predict_proba(test)[:,1]

In [ ]:
# csv 파일1 생성 (예시와 다른 형태)
pd.DataFrame({'index':test.index, 'y_pred':pred}).to_csv('0000.csv', index=False)

In [ ]:
# csv 파일2 생성
pd.DataFrame({'y_pred':pred}).reset_index().to_csv('1111.csv', index=False)

In [ ]:
# csv 파일1 확인
pd.read_csv('0000.csv')

,index,y_pred
0,0,0.149264
1,1,0.128597
2,2,0.125320
3,3,0.985559
4,4,0.153028
...,...,...
492,492,0.104853
493,493,0.983758
494,494,0.121372
495,495,0.965155


In [ ]:
# csv 파일2 확인
pd.read_csv('1111.csv')

,index,y_pred
0,0,0.149264
1,1,0.128597
2,2,0.125320
3,3,0.985559
4,4,0.153028
...,...,...
492,492,0.104853
493,493,0.983758
494,494,0.121372
495,495,0.965155


## 체점
- 수험자는 알 수 없는 부분임

In [ ]:
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/y_test.csv")
roc_auc_score(y_test, pred)

0.7875693000693

In [ ]:
# 당신의 score는?: 0.7875693000693